# ML Work

## Predicting total vaccinations administered per hundred and total people vaccinated per hundred based on happiness factors 

### Splitting Data

In [36]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# import and prepare data
happiness = pd.read_csv('happiness.csv')
vaccinations = pd.read_csv('country_vaccinations.csv')
vaccinations.head()
total_vaccinations = vaccinations.groupby('country').max()[['total_vaccinations_per_hundred', 'people_vaccinated_per_hundred']]
merged = pd.merge(happiness, total_vaccinations, left_on='location', right_on='country').dropna()
data = merged[['Ladder score','Logged GDP per capita',  'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity', 'Perceptions of corruption', 'Ladder score in Dystopia', 'Explained by: Log GDP per capita', 'Explained by: Social support',
       'Explained by: Healthy life expectancy',
       'Explained by: Freedom to make life choices',
       'Explained by: Generosity', 'Explained by: Perceptions of corruption']].values
data = StandardScaler().fit_transform(data)
target = merged[['total_vaccinations_per_hundred',
       'people_vaccinated_per_hundred']].values
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=.2, random_state=216)


### Linear Regression

In [37]:
# Training and Predicting model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
lin_model = LinearRegression().fit(X=train_data, y=train_target)
predicted = lin_model.predict(test_data)
print('MSE:', mean_squared_error(test_target, predicted))
print('r2:', r2_score(test_target, predicted))

MSE: 520.5692285637629
r2: 3.4892727518387456e-05


### K Nearest Neighbors

In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
knn = GridSearchCV(estimator=KNeighborsRegressor(), param_grid={'n_neighbors': range(1,30)})
knn.fit(X=train_data, y=train_target)
knn_predicted = knn.predict(test_data)
print(knn.best_params_)
print('MSE:', mean_squared_error(test_target, knn_predicted))
print('r2:', r2_score(test_target, knn_predicted))

{'n_neighbors': 2}
MSE: 466.31633124999985
r2: 0.0853212717459052


### Polynomial Features

In [39]:
from sklearn.preprocessing import PolynomialFeatures 
for i in range(2,7):
    deg = i
    poly_train_data = PolynomialFeatures(deg, include_bias=False).fit_transform(train_data)
    poly_model = LinearRegression()
    poly_model.fit(X=poly_train_data, y=train_target)
    poly_test_data=PolynomialFeatures(deg, include_bias=False).fit_transform(test_data)
    poly_predicted = poly_model.predict(poly_test_data)
    print('For Degree = ', deg)
    print('MSE:', mean_squared_error(test_target, poly_predicted))
    print('r2:', r2_score(test_target, poly_predicted))
    print('\n')

For Degree =  2
MSE: 5011.949212569036
r2: -9.61345800691978


For Degree =  3
MSE: 4479.891749552646
r2: -8.455276556870631


For Degree =  4
MSE: 911.102309222994
r2: -0.8704910390827151


For Degree =  5
MSE: 4115.653387715726
r2: -7.305441369510395


For Degree =  6
MSE: 8062.783731129151
r2: -16.00901805274353




### Convolutional Neural Network

In [40]:
#import
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [50]:
#convert to tensors
X_train = torch.from_numpy(train_data.astype(np.float32))
y_train = torch.from_numpy(train_target.astype(np.float32))
input_size = 14
output_size = 2

In [58]:
#define network
input_size = 14
output_size = 2

class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear = torch.nn.Linear(14, 2)  
        
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

learning_rate = 0.0001
l = nn.MSELoss()
model = NeuralNetwork()
optimizer = torch.optim.SGD(model.parameters(), lr =learning_rate )


In [61]:
#training

num_epochs = 10000 
for epoch in range(num_epochs):
    #forward feed
    y_pred = model(X_train.requires_grad_())

    #calculate the loss
    loss= l(y_pred, y_train)

    #backward propagation: calculate gradients
    loss.backward()

    #update the weights
    optimizer.step()

    #clear out the gradients from the last step loss.backward()
    optimizer.zero_grad()
    
    print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 53.27891540527344
epoch 1, loss 53.2788200378418
epoch 2, loss 53.278724670410156
epoch 3, loss 53.27862548828125
epoch 4, loss 53.27853012084961
epoch 5, loss 53.27843475341797
epoch 6, loss 53.2783317565918
epoch 7, loss 53.278236389160156
epoch 8, loss 53.27813720703125
epoch 9, loss 53.27804183959961
epoch 10, loss 53.27794647216797
epoch 11, loss 53.2778434753418
epoch 12, loss 53.277748107910156
epoch 13, loss 53.277652740478516
epoch 14, loss 53.27755355834961
epoch 15, loss 53.27745819091797
epoch 16, loss 53.27736282348633
epoch 17, loss 53.277259826660156
epoch 18, loss 53.27716064453125
epoch 19, loss 53.27707290649414
epoch 20, loss 53.27696990966797
epoch 21, loss 53.27687454223633
epoch 22, loss 53.27677917480469
epoch 23, loss 53.276676177978516
epoch 24, loss 53.27658462524414
epoch 25, loss 53.27648162841797
epoch 26, loss 53.27638626098633
epoch 27, loss 53.27629089355469
epoch 28, loss 53.276187896728516
epoch 29, loss 53.27609634399414
epoch 30, loss 5

epoch 274, loss 53.25252914428711
epoch 275, loss 53.25243377685547
epoch 276, loss 53.25233840942383
epoch 277, loss 53.25224304199219
epoch 278, loss 53.25215148925781
epoch 279, loss 53.25205612182617
epoch 280, loss 53.25196075439453
epoch 281, loss 53.25186538696289
epoch 282, loss 53.25177001953125
epoch 283, loss 53.25167465209961
epoch 284, loss 53.25157928466797
epoch 285, loss 53.25148391723633
epoch 286, loss 53.25138854980469
epoch 287, loss 53.25129318237305
epoch 288, loss 53.251197814941406
epoch 289, loss 53.251102447509766
epoch 290, loss 53.251007080078125
epoch 291, loss 53.250911712646484
epoch 292, loss 53.250816345214844
epoch 293, loss 53.25072479248047
epoch 294, loss 53.25062942504883
epoch 295, loss 53.25053405761719
epoch 296, loss 53.25043869018555
epoch 297, loss 53.250343322753906
epoch 298, loss 53.250247955322266
epoch 299, loss 53.250152587890625
epoch 300, loss 53.250057220458984
epoch 301, loss 53.249969482421875
epoch 302, loss 53.2498664855957
epoch

epoch 578, loss 53.2239875793457
epoch 579, loss 53.22389221191406
epoch 580, loss 53.22380447387695
epoch 581, loss 53.223716735839844
epoch 582, loss 53.22361373901367
epoch 583, loss 53.22352600097656
epoch 584, loss 53.22343063354492
epoch 585, loss 53.22334289550781
epoch 586, loss 53.22324752807617
epoch 587, loss 53.22315216064453
epoch 588, loss 53.223060607910156
epoch 589, loss 53.22297286987305
epoch 590, loss 53.22287368774414
epoch 591, loss 53.2227897644043
epoch 592, loss 53.222686767578125
epoch 593, loss 53.222591400146484
epoch 594, loss 53.222511291503906
epoch 595, loss 53.222408294677734
epoch 596, loss 53.222320556640625
epoch 597, loss 53.222232818603516
epoch 598, loss 53.222137451171875
epoch 599, loss 53.222042083740234
epoch 600, loss 53.22195053100586
epoch 601, loss 53.221858978271484
epoch 602, loss 53.22176742553711
epoch 603, loss 53.2216796875
epoch 604, loss 53.22158432006836
epoch 605, loss 53.22148895263672
epoch 606, loss 53.22140121459961
epoch 607

epoch 954, loss 53.1897087097168
epoch 955, loss 53.18962097167969
epoch 956, loss 53.18953323364258
epoch 957, loss 53.1894416809082
epoch 958, loss 53.18935012817383
epoch 959, loss 53.18925476074219
epoch 960, loss 53.189170837402344
epoch 961, loss 53.18907928466797
epoch 962, loss 53.18899154663086
epoch 963, loss 53.18890380859375
epoch 964, loss 53.188819885253906
epoch 965, loss 53.188724517822266
epoch 966, loss 53.188636779785156
epoch 967, loss 53.188541412353516
epoch 968, loss 53.188453674316406
epoch 969, loss 53.1883659362793
epoch 970, loss 53.18827819824219
epoch 971, loss 53.18818664550781
epoch 972, loss 53.18809509277344
epoch 973, loss 53.188011169433594
epoch 974, loss 53.18791580200195
epoch 975, loss 53.187828063964844
epoch 976, loss 53.187740325927734
epoch 977, loss 53.187644958496094
epoch 978, loss 53.187557220458984
epoch 979, loss 53.187469482421875
epoch 980, loss 53.1873779296875
epoch 981, loss 53.187286376953125
epoch 982, loss 53.18720245361328
epoch

epoch 1325, loss 53.15692901611328
epoch 1326, loss 53.15684127807617
epoch 1327, loss 53.1567497253418
epoch 1328, loss 53.15666580200195
epoch 1329, loss 53.156578063964844
epoch 1330, loss 53.156490325927734
epoch 1331, loss 53.15640640258789
epoch 1332, loss 53.15631866455078
epoch 1333, loss 53.15623474121094
epoch 1334, loss 53.15614318847656
epoch 1335, loss 53.15605163574219
epoch 1336, loss 53.15597915649414
epoch 1337, loss 53.1558837890625
epoch 1338, loss 53.155799865722656
epoch 1339, loss 53.15570831298828
epoch 1340, loss 53.15562057495117
epoch 1341, loss 53.15553665161133
epoch 1342, loss 53.15544891357422
epoch 1343, loss 53.15536117553711
epoch 1344, loss 53.155277252197266
epoch 1345, loss 53.155189514160156
epoch 1346, loss 53.15510177612305
epoch 1347, loss 53.15502166748047
epoch 1348, loss 53.15492630004883
epoch 1349, loss 53.15483474731445
epoch 1350, loss 53.15475845336914
epoch 1351, loss 53.1546630859375
epoch 1352, loss 53.15458297729492
epoch 1353, loss 5

...

epoch 8506, loss 52.66729736328125
epoch 8507, loss 52.66725158691406
epoch 8508, loss 52.66719055175781
epoch 8509, loss 52.667137145996094
epoch 8510, loss 52.66708755493164
epoch 8511, loss 52.66703414916992
epoch 8512, loss 52.66697692871094
epoch 8513, loss 52.66691970825195
epoch 8514, loss 52.666873931884766
epoch 8515, loss 52.66681671142578
epoch 8516, loss 52.66676330566406
epoch 8517, loss 52.66670608520508
epoch 8518, loss 52.666656494140625
epoch 8519, loss 52.666603088378906
epoch 8520, loss 52.66654968261719
epoch 8521, loss 52.6664924621582
epoch 8522, loss 52.666439056396484
epoch 8523, loss 52.66638946533203
epoch 8524, loss 52.66632843017578
epoch 8525, loss 52.66627883911133
epoch 8526, loss 52.66622543334961
epoch 8527, loss 52.66617202758789
epoch 8528, loss 52.66611862182617
epoch 8529, loss 52.66606140136719
epoch 8530, loss 52.66600799560547
epoch 8531, loss 52.665958404541016
epoch 8532, loss 52.66590881347656
epoch 8533, loss 52.66585159301758
epoch 8534, los

epoch 9084, loss 52.636714935302734
epoch 9085, loss 52.63665771484375
epoch 9086, loss 52.63661193847656
epoch 9087, loss 52.636558532714844
epoch 9088, loss 52.63650131225586
epoch 9089, loss 52.63645553588867
epoch 9090, loss 52.63640213012695
epoch 9091, loss 52.6363525390625
epoch 9092, loss 52.63629913330078
epoch 9093, loss 52.63624954223633
epoch 9094, loss 52.636192321777344
epoch 9095, loss 52.63614273071289
epoch 9096, loss 52.63609313964844
epoch 9097, loss 52.63603973388672
epoch 9098, loss 52.635986328125
epoch 9099, loss 52.63594055175781
epoch 9100, loss 52.63588333129883
epoch 9101, loss 52.63582992553711
epoch 9102, loss 52.635780334472656
epoch 9103, loss 52.63572692871094
epoch 9104, loss 52.635677337646484
epoch 9105, loss 52.6356201171875
epoch 9106, loss 52.63557434082031
epoch 9107, loss 52.635520935058594
epoch 9108, loss 52.635467529296875
epoch 9109, loss 52.63541793823242
epoch 9110, loss 52.6353645324707
epoch 9111, loss 52.635311126708984
epoch 9112, loss 

epoch 9420, loss 52.61939239501953
epoch 9421, loss 52.61935043334961
epoch 9422, loss 52.61929702758789
epoch 9423, loss 52.61924362182617
epoch 9424, loss 52.619197845458984
epoch 9425, loss 52.619144439697266
epoch 9426, loss 52.61909484863281
epoch 9427, loss 52.61903762817383
epoch 9428, loss 52.618988037109375
epoch 9429, loss 52.61893844604492
epoch 9430, loss 52.61888885498047
epoch 9431, loss 52.61884307861328
epoch 9432, loss 52.6187858581543
epoch 9433, loss 52.61873245239258
epoch 9434, loss 52.61868667602539
epoch 9435, loss 52.61863327026367
epoch 9436, loss 52.61858367919922
epoch 9437, loss 52.618526458740234
epoch 9438, loss 52.61848068237305
epoch 9439, loss 52.618431091308594
epoch 9440, loss 52.61837387084961
epoch 9441, loss 52.61832809448242
epoch 9442, loss 52.6182746887207
epoch 9443, loss 52.618228912353516
epoch 9444, loss 52.6181755065918
epoch 9445, loss 52.618125915527344
epoch 9446, loss 52.61807632446289
epoch 9447, loss 52.61802291870117
epoch 9448, loss

epoch 9700, loss 52.605220794677734
epoch 9701, loss 52.605167388916016
epoch 9702, loss 52.60512161254883
epoch 9703, loss 52.60506820678711
epoch 9704, loss 52.60502243041992
epoch 9705, loss 52.6049690246582
epoch 9706, loss 52.604915618896484
epoch 9707, loss 52.6048698425293
epoch 9708, loss 52.604820251464844
epoch 9709, loss 52.604766845703125
epoch 9710, loss 52.60471725463867
epoch 9711, loss 52.604671478271484
epoch 9712, loss 52.6046142578125
epoch 9713, loss 52.60456848144531
epoch 9714, loss 52.604515075683594
epoch 9715, loss 52.60446548461914
epoch 9716, loss 52.60441970825195
epoch 9717, loss 52.6043701171875
epoch 9718, loss 52.604312896728516
epoch 9719, loss 52.60426330566406
epoch 9720, loss 52.60421371459961
epoch 9721, loss 52.60416793823242
epoch 9722, loss 52.6041145324707
epoch 9723, loss 52.604061126708984
epoch 9724, loss 52.60401916503906
epoch 9725, loss 52.60396957397461
epoch 9726, loss 52.60391616821289
epoch 9727, loss 52.60386276245117
epoch 9728, loss

epoch 9987, loss 52.59092330932617
epoch 9988, loss 52.59086990356445
epoch 9989, loss 52.590824127197266
epoch 9990, loss 52.59077072143555
epoch 9991, loss 52.59072494506836
epoch 9992, loss 52.590675354003906
epoch 9993, loss 52.59062194824219
epoch 9994, loss 52.590572357177734
epoch 9995, loss 52.59053039550781
epoch 9996, loss 52.59047317504883
epoch 9997, loss 52.59042739868164
epoch 9998, loss 52.59037780761719
epoch 9999, loss 52.590328216552734


In [64]:
# Evalutation
nn_predicted = model(torch.from_numpy(test_data.astype(np.float32))).detach().numpy()
print('MSE:', mean_squared_error(test_target, nn_predicted))
print('r2:', r2_score(test_target, nn_predicted))

MSE: 479.0676618892576
r2: 0.0893414467083708


## Predicting happiness factors based on total vaccinations administered per hundred and total people vaccinated per hundred

### Splitting Data

In [7]:
target = merged[['Ladder score','Logged GDP per capita',  'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity', 'Perceptions of corruption', 'Ladder score in Dystopia', 'Explained by: Log GDP per capita', 'Explained by: Social support',
       'Explained by: Healthy life expectancy',
       'Explained by: Freedom to make life choices',
       'Explained by: Generosity', 'Explained by: Perceptions of corruption']].values
target = StandardScaler().fit_transform(data)
data = merged[['total_vaccinations_per_hundred',
       'people_vaccinated_per_hundred']].values
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=.2, random_state=216)

### Linear Regression

In [8]:
# Training and Predicting model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
model = LinearRegression().fit(X=train_data, y=train_target)
predicted = model.predict(test_data)
print('MSE:', mean_squared_error(test_target, predicted))
print('r2:', r2_score(test_target, predicted))

MSE: 1.4764039062843939
r2: -0.526276114382343


### K Nearest Neighbors

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
knn = GridSearchCV(estimator=KNeighborsRegressor(), param_grid={'n_neighbors': range(1,30)})
knn.fit(X=train_data, y=train_target)
knn_predicted = knn.predict(test_data)
print(knn.best_params_)
print('MSE:', mean_squared_error(test_target, knn_predicted))
print('r2:', r2_score(test_target, knn_predicted))

{'n_neighbors': 18}
MSE: 0.5960565996609892
r2: 0.3699128041077059


### Polynomial Features

In [10]:
from sklearn.preprocessing import PolynomialFeatures 
for i in range(2,10):
    deg = i
    poly_train_data = PolynomialFeatures(deg, include_bias=False).fit_transform(train_data)
    poly_model = LinearRegression()
    poly_model.fit(X=poly_train_data, y=train_target)
    poly_test_data=PolynomialFeatures(deg, include_bias=False).fit_transform(test_data)
    poly_predicted = poly_model.predict(poly_test_data)
    print('For Degree = ', deg)
    print('MSE:', mean_squared_error(test_target, poly_predicted))
    print('r2:', r2_score(test_target, poly_predicted))
    print('\n')

For Degree =  2
MSE: 81.9400242945651
r2: -82.99236974709423


For Degree =  3
MSE: 6071.183247913994
r2: -7336.014118845635


For Degree =  4
MSE: 919433.6248495538
r2: -1039326.1138365762


For Degree =  5
MSE: 1767180753.19922
r2: -2460787757.7068987


For Degree =  6
MSE: 16893327929231.135
r2: -17531045611489.676


For Degree =  7
MSE: 2.2033770883780054e+17
r2: -3.1420755999491776e+17


For Degree =  8
MSE: 8.290509126079196e+19
r2: -9.755071340627401e+19


For Degree =  9
MSE: 2.9015136068934026e+21
r2: -3.1514782677558116e+21


